# Project Description:

This assignment focuses on building and evaluating different classifiers for breast cancer classification using the breast cancer dataset. The classifiers implemented include Bagging, Boosting, and a Dummy Classifier.

#### data science pipeline

1. **Understanding the Dataset**: understanding of the dataset by reviewing the dataset documentation and exploring the features.

2. **Load the Data**: Reading the dataset file (`breast-cancer.csv`) and load it.

3. **Exploratory Analysis**: Performing exploratory data analysis to reach insights into the dataset. This include visualizations and statistical summaries.

4. **Data Preprocessing**: Applying necessary preprocessing steps such as handling missing values, handling skewness, and normalizing the data.

5. **Modeling**: Creating and training different classifiers. Implement Bagging classifiers with various estimators and a Dummy Classifier.

6. **Evaluation**: Evaluating the performance of the classifiers using appropriate evaluation metrics such as accuracy, precision, recall, and F1-score. Compare the performance of different models and analyze why some methods perform better than others. Experiment with different configurations for Bagging and Boosting models.

7. **Visualization**: Visualization the results and evaluation metrics using appropriate plots. Plot the Precision-Recall Curve for decision tree and random forest classifiers. Additionally, generate feature importance plots for the random forest classifier.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import yaml
import seaborn as sns

# Set random seed for reproducibility
random_seed = 42

#### loading data & exploratory analysis

In this part, the breast cancer dataset is read and the first few rows are displayed to have overview of data and types of them.

After that,  dataset's information including column names, data types, and non-null counts are printed to achieve more detail of breast cancer data status and also the distribution of the target variable 'diagnosis' which indicates the counts of malignant (M) and benign (B) is checked.

In [ ]:
# Import the data
configPath = 'config.yaml'

# Read the yaml data from the file
with open(configPath, 'r') as file:
    configData = yaml.safe_load(file)

df = pd.read_csv(configData["breast_cancer_path"])

# Display the first few rows of the dataset
print(df.head())

In [ ]:
# Getting an overview of the dataset
print(df.info())

As above results show, it seems there is no null values in the data, but in the following code we will double check it with static methods.

In [ ]:
# Check the distribution of target variable
print(df['diagnosis'].value_counts())

# Check for missing values
print(df.isnull().sum())

The diagnosis columns involves 357 B class and 212 M class. and as previouse part showed there is no non-value feature.

In [ ]:
# Visualization the distribution of features
import matplotlib.pyplot as plt

df.drop(['id'], axis=1).hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()

The distrbution of features expect 'id' field are checked and regarding the right tail it seems that the data are not normalized.

Id field is not important field in this assignment and don't give significant data for breast cancer status in this assignment.

#### preprocessing data

As the 'diagnosis' column is an categorical, it is encoded for the classifiers to work with numeric labels.

Also, data for furture calculations are splitted to dependent(/target) variable ('diagnosis column) and independent (features) variable.



Finally, the missing value are handled with the mean value.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the diagnosis column (Malignant = 1, Benign = 0)
label_encoder = LabelEncoder()
df['diagnosis'] = label_encoder.fit_transform(df['diagnosis'])

# Split the data into features (X) and target(dependent) variable (y)
X = df.drop(['id', 'diagnosis'], axis=1)
y = df['diagnosis']

# handling missing values (replace them with mean)
X = X.fillna(X.mean())

In this step, the code identifies features with skewness above a threshold of 0.7. Skewness indicates the asymmetry of the distribution of a feature. The code then applies a power transformation to these skewed features to make their distributions more symmetric and closer to a normal distribution. 

After that, the features using standard scaling, which ensures that each feature has a mean of 0 and a standard deviation of 1. This helps improve the performance of many machine learning algorithms that are sensitive to the scale of features. 

The following part is done because of the result of exploratory analysis that showed there is a different variance between data.

Finally, the test and train data are created based on prepared data for the rest analysis of breast cancer dataset.

In [ ]:
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

# Calculating skewness of the features
skewness = X.skew()

# Selecting features with skewness above a threshold (0.7)
skewed_features = skewness[abs(skewness) > 0.7].index

# Applying power transformation 
power_transformer = PowerTransformer()
X_skewed = X[skewed_features].copy()
X_skewed_transformed = power_transformer.fit_transform(X_skewed)

# Replacing the original skewed features with the transformed features
X[skewed_features] = X_skewed_transformed

# Normalizing the features using standard scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=random_seed)

#### Creating the bagging, boosting, and dummy classifiers

Here, different instances of Bagging and AdaBoost classifiers are created with varying numbers of estimators. The BaggingClassifier combines multiple base estimators to improve model performance, while AdaBoostClassifier adapts its focus on samples that were misclassified by the previous estimator. A DummyClassifier is used as a baseline that always predicts the most frequent class.

Increasing the number of estimators can improve performance, but it increases computation time, as well; Because of this in the following just 15, 10 and 5 values is used for estimators.

In [ ]:
# Bagging Classifier with different estimator
bagging_clf_15 = BaggingClassifier(n_estimators=15, random_state=random_seed)
bagging_clf_10 = BaggingClassifier(n_estimators=10, random_state=random_seed)
bagging_clf_5 = BaggingClassifier(n_estimators=5, random_state=random_seed)

# AdaBoost Classifier with different estimator
adaboost_clf_15 = AdaBoostClassifier(n_estimators=15, random_state=random_seed)
adaboost_clf_10 = AdaBoostClassifier(n_estimators=10, random_state=random_seed)
adaboost_clf_5 = AdaBoostClassifier(n_estimators=5, random_state=random_seed)

# Dummy Classifier as a baseline
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=random_seed)

In the following section, the classifiers are trained with the training data and then used to predict the labels of the test data to used in the evaluatation models' part.

In [ ]:
# Train and evaluate Bagging Classifier
bagging_clf_15.fit(X_train, y_train)
bagging_pred_15 = bagging_clf_15.predict(X_test)

bagging_clf_10.fit(X_train, y_train)
bagging_pred_10 = bagging_clf_10.predict(X_test)

bagging_clf_5.fit(X_train, y_train)
bagging_pred_5 = bagging_clf_5.predict(X_test)

# Train and evaluate AdaBoost Classifier
adaboost_clf_15.fit(X_train, y_train)
adaboost_pred_15 = adaboost_clf_15.predict(X_test)

adaboost_clf_10.fit(X_train, y_train)
adaboost_pred_10 = adaboost_clf_10.predict(X_test)

adaboost_clf_5.fit(X_train, y_train)
adaboost_pred_5 = adaboost_clf_5.predict(X_test)

# Train and evaluate Dummy Classifier
dummy_clf.fit(X_train, y_train)
dummy_pred = dummy_clf.predict(X_test)


#### Method Evaluation

The predictions calculated in the above section are used to evaluate the models' performance using various evaluation metrics like accuracy, precision, recall, and F1-score.

In [ ]:
# Define evaluation metrics
metrics = {
    'Accuracy': accuracy_score,
    'Precision': precision_score,
    'Recall': recall_score,
    'F1-score': f1_score
}

#this function evaluate the specific model based on defined metrics and prediction set
def evaluate_model(model, predictions):
    eval_results = {}
    for metric_name, metric_func in metrics.items():
        if model.classes_[0] == 'B':
            label_mapping = {'B': 0, 'M': 1}
            y_test_mapped = y_test.map(label_mapping)
            predictions_mapped = [label_mapping[pred] for pred in predictions]
        else:
            y_test_mapped = y_test
            predictions_mapped = predictions
        eval_results[metric_name] = metric_func(y_test_mapped, predictions_mapped)
    return eval_results

In [ ]:
# Evaluating Bagging Classifier
bagging_results_15 = evaluate_model(bagging_clf_15, bagging_pred_15)
bagging_results_10 = evaluate_model(bagging_clf_10, bagging_pred_10)
bagging_results_5 = evaluate_model(bagging_clf_5, bagging_pred_5)

# Evaluating AdaBoost Classifier
adaboost_results_15 = evaluate_model(adaboost_clf_15, adaboost_pred_15)
adaboost_results_10 = evaluate_model(adaboost_clf_10, adaboost_pred_10)
adaboost_results_5 = evaluate_model(adaboost_clf_5, adaboost_pred_5)

# Evaluating Dummy Classifier
dummy_results = evaluate_model(dummy_clf, dummy_pred)

# Displaying the results
print('Bagging Classifier Results:')
print("15 estimator", bagging_results_15)
print("10 estimator", bagging_results_10)
print("5 estimator", bagging_results_5)
print('\nAdaBoost Classifier Results:')
print("15  estimator", adaboost_results_15)
print("10 estimator", adaboost_results_10)
print("5  estimator", adaboost_results_5)
print('\nDummy Classifier Results:')
print(dummy_results)


As Bagging Classifier and the AdaBoost Classifier evaluation results based on all metrics has a high score (over 90%), it shows that both are performing well, and also the AdaBoost Classifier having slightly better results, especially when using a smaller number of estimators (5). 

The Dummy Classifier, on the other hand, is performing poorly and isn't providing meaningful predictions for given breast cancer data. It reaches an accuracy of about 62.28%, but it has very low precision, recall, and F1-score, all being 0.

#### Visualization results

In the following, the results of each model with estimators are plotted with confusion matrix and ROC curve.

The confusion matrix shows the counts of true positive, true negative, false positive, and false negative predictions of breast cancer data, while the ROC curve illustrates the trade-off between true positive rate and false positive rate.

As in the above section, Dummy Classifier didn't reach to the proper results in the following is not considered.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve

# ploting the result of Confusion Matrix

class_names = ['Benign', 'Malignant']

fig, axes = plt.subplots(3, 2, figsize=(12, 18))

# 15 estimators
plot_confusion_matrix(bagging_clf_15, X_test, y_test, display_labels=class_names, cmap=plt.cm.Blues, ax=axes[0, 0])
axes[0, 0].set_title("Bagging (15 Estimators)")
plot_confusion_matrix(adaboost_clf_15, X_test, y_test, display_labels=class_names, cmap=plt.cm.Blues, ax=axes[0, 1])
axes[0, 1].set_title("Adaboost (15 Estimators)")

# 10 estimators
plot_confusion_matrix(bagging_clf_10, X_test, y_test, display_labels=class_names, ax=axes[1, 0])
axes[1, 0].set_title("Bagging (10 Estimators)")
plot_confusion_matrix(adaboost_clf_10, X_test, y_test, display_labels=class_names, ax=axes[1, 1])
axes[1, 1].set_title("Adaboost (10 Estimators)")

# 5 estimators
plot_confusion_matrix(bagging_clf_5, X_test, y_test, display_labels=class_names, cmap=plt.cm.Greens, ax=axes[2, 0])
axes[2, 0].set_title("Bagging (5 Estimators)")
plot_confusion_matrix(adaboost_clf_5, X_test, y_test, display_labels=class_names, cmap=plt.cm.Greens, ax=axes[2, 1])
axes[2, 1].set_title("Adaboost (5 Estimators)")

# Adjust layout
plt.tight_layout()
plt.show()

As shown in the above figures both models with all estimators works well (all have error type 1 and 2 under 5) but like the previouse section the Adaboost works better than baggin especially with 5 estimators which has 2 FN and 0 FP.

FN: means model predicted 'Benign' but the actual result was 'Malignant'

FP: means model predicted 'Malignant' but the actual result was 'Benign'

In [ ]:
# ploting the result of roc curve 

fig, axes = plt.subplots(3, 2, figsize=(12, 18))

# 15 estimators
plot_roc_curve(bagging_clf_15, X_test, y_test, ax=axes[0, 0])
axes[0, 0].set_title("Bagging (15 Estimators)")
plot_roc_curve(adaboost_clf_15, X_test, y_test, ax=axes[0, 1])
axes[0, 1].set_title("Adaboost (15 Estimators)")

# 10 estimators
plot_roc_curve(bagging_clf_10, X_test, y_test, ax=axes[1, 0])
axes[1, 0].set_title("Bagging (10 Estimators)")
plot_roc_curve(adaboost_clf_10, X_test, y_test, ax=axes[1, 1])
axes[1, 1].set_title("Adaboost (10 Estimators)")

# 5 estimators
plot_roc_curve(bagging_clf_5, X_test, y_test, ax=axes[2, 0])
axes[2, 0].set_title("Bagging (5 Estimators)")
plot_roc_curve(adaboost_clf_5, X_test, y_test, ax=axes[2, 1])
axes[2, 1].set_title("Adaboost (5 Estimators)")

# Adjust layout
plt.tight_layout()
plt.show()

Both methods with their estimators have almost same results just AdaBoost Classifier with 5 estimators covers its Area under the curve (AUC) more than others in the almost 0.99 scores. So it shows this type of model slightly works better than others. 

#### Feature Importance Plot

As in the previouse part the AdaBoost classifier with 5 estimators works better than others in this section, feature importances are calculated for this model and Bagging with 5 estimators as well to compare with each other and see the differences in importance features of breast cancer.

Feature importance indicates how much each feature contributes to the model's predictions. The code then sorts and plots these importances for each estimator.

In [ ]:
import numpy as np 

# for bagging
importances_5 = bagging_clf_5.estimators_[0].feature_importances_
feature_names = X.columns # Getting feature names
indices_5 = np.argsort(importances_5)[::-1] # Sorting feature importances in descending order

plt.figure(figsize=(10, 6))
plt.bar(range(X.shape[1]), importances_5[indices_5])
plt.xticks(range(X.shape[1]), feature_names[indices_5], rotation=90)
plt.title('Feature Importance of Adaboost with 5 estimators')
plt.show()

#for adaboost
importances_5 = adaboost_clf_5.estimators_[0].feature_importances_

# Get feature names
feature_names = X.columns

# Sort feature importances in descending order
indices_5 = np.argsort(importances_5)[::-1]

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(X.shape[1]), importances_5[indices_5])
plt.xticks(range(X.shape[1]), feature_names[indices_5], rotation=90)
plt.title('Feature Importance of Adaboost with 5 estimators')
plt.show()

As shown in the above the main feature that impact on the model for prediction of breast cancer in Adaboost classifier with 5 estimator is "concave points_mean", meanwhile for bagging with the same estimators there are 8 features that have main role in prediction and the most important is "perimeter_worst" feature.